In [1]:
!pip install -q datasets
!huggingface-cli login

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12=

In [3]:
from datasets import load_dataset
import re

# Step 1: Load your custom dataset from Hugging Face
dataset = load_dataset('medss19/home_remedies_dataset', split='train')

# Step 2: Shuffle and slice
dataset = dataset.shuffle(seed=42)

# Step 3: Transform the dataset similar to how LLaMA2 expects it
def transform_remedy(example):
    user_prompt = f"I am suffering from {example['Health Issue']}. Please suggest a home remedy."
    assistant_response = (
        f"Here is a suggested remedy:\n\n"
        f"**Item:** {example['Name of Item']}\n"
        f"**Remedy:** {example['Home Remedy']}\n"
        f"**Recommended Yoga:** {example['Yogasan']}\n"
    )
    return {
        "text": f"<s>[INST] {user_prompt} [/INST] {assistant_response} </s>"
    }

# Step 4: Apply the transformation
transformed_dataset = dataset.map(transform_remedy)

# Step 5: View a sample
print(transformed_dataset[0])


Map:   0%|          | 0/115 [00:00<?, ? examples/s]

{'Name of Item': 'DHANIA ( Conundrum sativum Linn., Dhanyaka)', 'Health Issue': 'Indigestion', 'Home Remedy': '20 ml. decoction prepared from 5 gm. coarse powder with a pinch of ginger powder thrice a day.', 'Yogasan': 'https://bostcdn.bostinnovation.com/which_exercise_helps_gastric_problem.jpg', 'text': '<s>[INST] I am suffering from Indigestion. Please suggest a home remedy. [/INST] Here is a suggested remedy:\n\n**Item:** DHANIA ( Conundrum sativum Linn., Dhanyaka)\n**Remedy:** 20 ml. decoction prepared from 5 gm. coarse powder with a pinch of ginger powder thrice a day.\n**Recommended Yoga:** https://bostcdn.bostinnovation.com/which_exercise_helps_gastric_problem.jpg\n </s>'}


In [4]:
transformed_dataset.push_to_hub("medss19/formatted-home-remedies")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/medss19/formatted-home-remedies/commit/c85d969ba553edcbe80e6dba0e0b9449894ad192', commit_message='Upload dataset', commit_description='', oid='c85d969ba553edcbe80e6dba0e0b9449894ad192', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/medss19/formatted-home-remedies', endpoint='https://huggingface.co', repo_type='dataset', repo_id='medss19/formatted-home-remedies'), pr_revision=None, pr_num=None)